In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Chọn mẫu ngẫu nhiên

In [21]:
import json
import random

def read_json_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return data




def get_random_samples(input_list, num_samples=20):
    """
    Get random samples from the input list.

    Args:
    input_list (list): The list to sample from.
    num_samples (int): The number of samples to return. Default is 20.

    Returns:
    list: A list of random samples.
    """
    if len(input_list) < num_samples:
        raise ValueError(f"Input list must have at least {num_samples} elements.")

    return random.sample(input_list, num_samples)
file_path = '/content/gdrive/My Drive/llama3_data/DuIE2.0/duie_dev.json'
json_data = read_json_file(file_path)

last_samples = get_random_samples(json_data)

# In ra các mẫu cuối cùng
for i, sample in enumerate(last_samples, 1):
    print(f"Sample {i}:")
    print(f"Text: {sample['text']}")
    print("SPO List:")
    for spo in sample['spo_list']:
        print(f"  Subject: {spo['subject']}")
        print(f"  Predicate: {spo['predicate']}")
        print(f"  Object: {spo['object']['@value']}")
        print(f"  Subject Type: {spo['subject_type']}")
        print(f"  Object Type: {spo['object_type']['@value']}")
        print()
    print("-" * 50)

Sample 1:
Text: 麟儿姐姐可能早就想问Simon:蔡国权有没有一首歌能和阿伦联系上的呢?答案是有的.这是一首蔡国权与阿伦对唱的歌曲."创造命运"的曲风大家是熟悉的.这首"风中劲草"有异曲同工的效果.都是教会大家一些做人的道理.Simon比较喜欢新的那个版本.原因有两个:第一是由蔡国权编曲.第二两人在2004年的合作中唱功都有了进步.很有成熟男人的味道.
SPO List:
  Subject: 创造命运
  Predicate: 作曲
  Object: 蔡国权
  Subject Type: 歌曲
  Object Type: 人物

--------------------------------------------------
Sample 2:
Text: 吉林省蛟河林业实验区管理局（简称吉林林业实验局），始建于1956年，现为吉林省林业厅直属正处级事业单位，承担着全省林业生态建设、教学研究、科技推广等试验示范任务
SPO List:
  Subject: 吉林省蛟河林业实验区管理局
  Predicate: 简称
  Object: 吉林林业实验局
  Subject Type: 机构
  Object Type: Text

  Subject: 吉林省蛟河林业实验区管理局
  Predicate: 成立日期
  Object: 1956年
  Subject Type: 机构
  Object Type: Date

--------------------------------------------------
Sample 3:
Text: 下一秒高能预警 没想到吧 第一季没想到吧：王祖蓝被观众嫌弃，下一秒直接出杀手锏，观众感动到哭简介：张绍刚、李诞、池子的“相聚”让人联想到《吐槽大会》，然而《没想到吧》却不是卫视版的“吐槽大会”，节目以“分享快乐”为主题，四位主持人将通过与观众的互动让他们感受到快乐，节目组致力于令主持人、明星嘉宾和观众共同度过一个全民参与、合家欢乐的夜晚
SPO List:
  Subject: 没想到吧
  Predicate: 主持人
  Object: 李诞
  Subject Type: 电视综艺
  Object Type: 人物

  Subject: 吐槽大会
  Predicate: 嘉宾


##hàm prompt tiếng trung

In [4]:

def create_prompt(text):
  eng_prompt = f"""Given the following text, extract the relationships between literary works, authors, and TV/film adaptations. Present the results in the following JSON format:

  {{
    "spo_list": [
      {{
        "predicate": "Relationship Type",
        "object_type": {{"@value": "Type of Object"}}
        "subject_type": "Type of Subject",
        "object": {{"@value": "Related Entity"}},
        "subject": "Work Name",
      }}
    ]
  }}
  Important: All values in the output, including predicates and objects, must be in Chinese. Do not use English for any field.

  Do not include predictions based on the text, just extract what is explicitly present in the text.


  Here's an example:

  Text: "《步步惊心》改编自著名作家桐华的同名清穿小说《甄嬛传》改编自流潋紫所著的同名小说电视剧《何以笙箫默》改编自顾漫同名小说《花千骨》改编自fresh果果同名小说《裸婚时代》是月影兰析创作的一部情感小说《琅琊榜》是根据海宴同名网络小说改编电视剧《宫锁心玉》，又名《宫》《雪豹》，该剧改编自网络小说《特战先驱》《我是特种兵》由红遍网络的小说《最后一颗子弹留给我》改编电视剧《来不及说我爱你》改编自匪我思存同名小说《来不及说我爱你》"

  Extracted Relationships:
  {{
    "spo_list": [
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "顾漫"}}, "subject": "何以笙箫默"}},
      {{"predicate": "改编自", "object_type": {{"@value": "作品"}}, "subject_type": "影视作品", "object": {{"@value": "最后一颗子弹留给我"}}, "subject": "我是特种兵"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "桐华"}}, "subject": "步步惊心"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "流潋紫"}}, "subject": "甄嬛传"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "fresh果果"}}, "subject": "花千骨"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "月影兰析"}}, "subject": "裸婚时代"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "海宴"}}, "subject": "琅琊榜"}},
      {{"predicate": "改编自", "object_type": {{"@value": "作品"}}, "subject_type": "影视作品", "object": {{"@value": "特战先驱"}}, "subject": "雪豹"}},
      {{"predicate": "改编自", "object_type": {{"@value": "作品"}}, "subject_type": "影视作品", "object": {{"@value": "来不及说我爱你"}}, "subject": "来不及说我爱你"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "匪我思存"}}, "subject": "来不及说我爱你"}}
      ]
  }}
  Now, please extract the relationships from the following text:

  Text: {text}

  Extracted Relationships:"""

  prompt = f"""
  根据以下文本，提取文学作品、作者和电视/电影改编之间的关系。以下列JSON格式呈现结果：

  {{
    "spo_list": [
      {{
        "predicate": "关系类型",
        "object_type": {{"@value": "对象类型"}},
        "subject_type": "主题类型",
        "object": {{"@value": "相关实体"}},
        "subject": "作品名称",
      }}
    ]
  }}

  重要提示：输出中的所有值，包括谓语和对象，都必须使用中文。

  不要包含基于文本的预测，只提取文本中明确存在的内容。

  以下是一个示例：

  文本："《步步惊心》改编自著名作家桐华的同名清穿小说《甄嬛传》改编自流潋紫所著的同名小说电视剧《何以笙箫默》改编自顾漫同名小说《花千骨》改编自fresh果果同名小说《裸婚时代》是月影兰析创作的一部情感小说《琅琊榜》是根据海宴同名网络小说改编电视剧《宫锁心玉》，又名《宫》《雪豹》，该剧改编自网络小说《特战先驱》《我是特种兵》由红遍网络的小说《最后一颗子弹留给我》改编电视剧《来不及说我爱你》改编自匪我思存同名小说《来不及说我爱你》"

  提取的关系：
  {{
    "spo_list": [
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "顾漫"}}, "subject": "何以笙箫默"}},
      {{"predicate": "改编自", "object_type": {{"@value": "作品"}}, "subject_type": "影视作品", "object": {{"@value": "最后一颗子弹留给我"}}, "subject": "我是特种兵"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "桐华"}}, "subject": "步步惊心"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "流潋紫"}}, "subject": "甄嬛传"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "fresh果果"}}, "subject": "花千骨"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "月影兰析"}}, "subject": "裸婚时代"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "海宴"}}, "subject": "琅琊榜"}},
      {{"predicate": "改编自", "object_type": {{"@value": "作品"}}, "subject_type": "影视作品", "object": {{"@value": "特战先驱"}}, "subject": "雪豹"}},
      {{"predicate": "改编自", "object_type": {{"@value": "作品"}}, "subject_type": "影视作品", "object": {{"@value": "来不及说我爱你"}}, "subject": "来不及说我爱你"}},
      {{"predicate": "作者", "object_type": {{"@value": "人物"}}, "subject_type": "图书作品", "object": {{"@value": "匪我思存"}}, "subject": "来不及说我爱你"}}
      ]
  }}

  现在，请从以下文本中提取关系：

  文本：{text}

  提取的关系：
  """
  return prompt


In [5]:
!pip install accelerate openai tiktoken transformers torch  retry bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "unsloth/llama-3-8b-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)






tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [7]:
!pip install translate

##Xử lý đầu ra

In [9]:
import re
def ensure_chinese_values(data):
    for item in data:
        item['predicate'] = translate_to_chinese(item['predicate'])
        item['object']['@value'] = translate_to_chinese(item['object']['@value'])
        item['object_type']['@value'] = translate_to_chinese(item['object_type']['@value'])
    return data


def parse_llm_output(output):
    # Tìm phần JSON trong đầu ra của LLM
    json_start = output.find('{')
    json_end = output.rfind('}') + 1
    json_str = output[json_start:json_end]

    try:
        parsed_json = json.loads(json_str)
        return parsed_json['spo_list']
    except json.JSONDecodeError:
        print("Error parsing JSON from LLM output")
        return []


## Hàm đánh giá kết quả

In [15]:
from difflib import SequenceMatcher
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from scipy.spatial.distance import cosine

semantic_check_model_name = "bert-base-uncased"
semantic_check_tokenizer = AutoTokenizer.from_pretrained(semantic_check_model_name)
semantic_check_model = AutoModel.from_pretrained(semantic_check_model_name)

def get_word_embedding(word):
    inputs = semantic_check_tokenizer(word, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = semantic_check_model(**inputs)
    return outputs.last_hidden_state[0][0].numpy()

def semantic_similarity(word1, word2):
    emb1 = get_word_embedding(word1)
    emb2 = get_word_embedding(word2)
    return 1 - cosine(emb1, emb2)

def relationships_match(rel1, rel2, similarity_threshold=0.8):
    # Check semantic similarity for predicate, object_type, and subject_type
    predicate_similarity = semantic_similarity(rel1["predicate"], rel2["predicate"])
    object_type_similarity = semantic_similarity(rel1["object_type"]["@value"], rel2["object_type"]["@value"])
    subject_type_similarity = semantic_similarity(rel1["subject_type"], rel2["subject_type"])

    # Check exact match for object and subject
    object_match = rel1["object"]["@value"] == rel2["object"]["@value"]
    subject_match = rel1["subject"] == rel2["subject"]

    # Return True if all conditions are met
    return (predicate_similarity >= similarity_threshold and
            object_type_similarity >= similarity_threshold and
            subject_type_similarity >= similarity_threshold and
            object_match and subject_match)

def compare_relationship_lists(predicted, ground_truth, similarity_threshold=0.8):
    TP = 0
    FP = 0
    FN = 0

    for pred in predicted:
        if any(relationships_match(pred, truth, similarity_threshold) for truth in ground_truth):
            TP += 1
        else:
            FP += 1

    for truth in ground_truth:
        if not any(relationships_match(truth, pred, similarity_threshold) for pred in predicted):
            FN += 1

    return TP, FP, FN

def calculate_f1(TP, FP, FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

def evaluate_output(llm_results, ground_truth):
    TP, FP, FN = compare_relationship_lists(llm_results, ground_truth)
    return calculate_f1(TP, FP, FN)


## Chạy từng mẫu và kết quả

In [19]:
def run_each_samples(sample):
  text = sample['text']
  ground_truth = sample['spo_list']
  prompt = create_prompt(text)
  #print(prompt)
  messages = [
    {"role": "system", "content": prompt },
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]
  outputs = model.generate(
    input_ids,
    max_new_tokens=3000,
    eos_token_id=terminators,
    do_sample=False,
    temperature=0.0,
  )
  response = outputs[0][input_ids.shape[-1]:]
  llm_output = tokenizer.decode(response, skip_special_tokens=True)
  print(llm_output)
  llm_results = parse_llm_output(llm_output)
  print(llm_results)
  print(ground_truth)
  return evaluate_output(llm_results, ground_truth)



In [22]:
total_precision = 0
total_recall = 0
total_f1 = 0

for i, sample in enumerate(last_samples, 1):
    precision, recall, f1 = run_each_samples(sample)
    total_precision += precision
    total_recall += recall
    total_f1 += f1
    print(total_precision)
    print(total_recall)
    print(total_f1)
total_precision /= len(last_samples)
total_recall /= len(last_samples)
total_f1 /= len(last_samples)

print(f"Average Precision: {total_precision:.4f}")
print(f"Average Recall: {total_recall:.4f}")
print(f"Average F1 Score: {total_f1:.4f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "spo_list": [
    {"predicate": "作者", "object_type": {"@value": "人物"}, "subject_type": "音乐作品", "object": {"@value": "蔡国权"}, "subject": "风中劲草"},
    {"predicate": "作者", "object_type": {"@value": "人物"}, "subject_type": "音乐作品", "object": {"@value": "蔡国权"}, "subject": "创造命运"}
  ]
}
[{'predicate': '作者', 'object_type': {'@value': '人物'}, 'subject_type': '音乐作品', 'object': {'@value': '蔡国权'}, 'subject': '风中劲草'}, {'predicate': '作者', 'object_type': {'@value': '人物'}, 'subject_type': '音乐作品', 'object': {'@value': '蔡国权'}, 'subject': '创造命运'}]
[{'predicate': '作曲', 'object_type': {'@value': '人物'}, 'subject_type': '歌曲', 'object': {'@value': '蔡国权'}, 'subject': '创造命运'}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


0.5
1.0
0.6666666666666666


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

7.7
7.166666666666666
6.666666666666666


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "spo_list": []
}

（无关系提取）
[]
[{'predicate': '导演', 'object_type': {'@value': '人物'}, 'subject_type': '影视作品', 'object': {'@value': '叶鸿伟'}, 'subject': '鲁冰花'}]
7.7
7.166666666666666
6.666666666666666


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "spo_list": [
    {
      "predicate": "关系类型",
      "object_type": {"@value": "对象类型"},
      "subject_type": "主题类型",
      "object": {"@value": "相关实体"},
      "subject": "作品名称"
    }
  ]
}

文本：停播事件台湾综艺节目《康熙来了》再次遭遇信任危机，昨天，知名娱乐圈爆料人“扒皮王”在自己的微博中爆料称，《康熙来了》的灵魂人物之一蔡康永将离开《康熙来了》，转战大陆市场

提取的关系：
{
  "spo_list": [
    {"predicate": "关系", "object_type": {"@value": "人物"}, "subject_type": "电视节目", "object": {"@value": "蔡康永"}, "subject": "康熙来了"}
  ]
}
Error parsing JSON from LLM output
[]
[{'predicate': '主持人', 'object_type': {'@value': '人物'}, 'subject_type': '电视综艺', 'object': {'@value': '蔡康永'}, 'subject': '康熙来了'}]
7.7
7.166666666666666
6.666666666666666
{
  "spo_list": []
}
[]
[{'predicate': '所在城市', 'object_type': {'@value': '城市'}, 'subject_type': '景点', 'object': {'@value': '北京'}, 'subject': '龙泉寺'}]
7.7
7.166666666666666
6.666666666666666
Average Precision: 0.3850
Average Recall: 0.3583
Average F1 Score: 0.3333
